In [ ]:
import os, json, glob, datetime, gc, re, time

from bs4 import BeautifulSoup

from urllib import parse
import sqlalchemy as sa
import pyodbc
import psycopg2

In [ ]:
os.chdir(r"D:\GitHub\repository\twitter\scrape\dump\newyearresolution_2020istheyear_#resolutions2020")

In [ ]:
creds_file = r"D:\personal\creds\psql\db_creds.json"
with open(creds_file) as f:
    creds = json.load(f)

uid = creds["uid"]
pw = creds["password"]
host = "localhost"
port = 5432
db = "twitscrape"

engine = sa.create_engine(f"postgresql+psycopg2://{uid}:{pw}@{host}:{port}/{db}")

In [ ]:
schema = "twitter"
tbl_name = "ny2020"

engine.execute(f"CREATE SCHEMA IF NOT EXISTS {schema}")
engine.execute(f"DROP TABLE IF EXISTS {schema}.{tbl_name}")

stmt = f"""
CREATE TABLE {schema}.{tbl_name}(
    datetime        timestamp NOT NULL,
    id_tweet        text NOT NULL,
    tweet_body      text,
    source          text,
    hashtags        text[],
    id_user         text,
    name_user       text,
    id_recipient    text,
    name_recipient  text,
    id_mentions     text[],
    name_mentions   text[],
    geo             double precision[],
    coordinates     double precision[],
    id_place        text,
    name_place      text,
    country         text,
    favorite_count  integer,
    quote_count     integer,
    reply_count     integer,
    retweet_count   integer,
    retweeted       boolean
)
"""
engine.execute(stmt)


stmt = f"""
SELECT
    column_name
FROM
    INFORMATION_SCHEMA.columns
WHERE
    table_schema = '{schema}'
    AND
    table_name = '{tbl_name}'
"""
fetch_cols = engine.execute(stmt).fetchall()

In [ ]:
col_names = ",".join([i[0] for i in fetch_cols])

In [ ]:
for file in os.listdir():
    with open(file, "r") as f:
        t = json.load(f)
    
    VAL_tweet_time = datetime.datetime.utcfromtimestamp(int(t["timestamp_ms"])/1000)
    VAL_id_tweet = t["id_str"]
    
    if t["truncated"] is True:
        VAL_tweet_body = t["extended_tweet"]["full_text"]
    elif "retweeted_status" in t.keys():
        if "extended_tweet" in t["retweeted_status"].keys():
            VAL_tweet_body = t["retweeted_status"]["extended_tweet"]["full_text"]
        else:
            VAL_tweet_body = t["retweeted_status"]["text"]
    else:
        VAL_tweet_body = t["text"]
    
    VAL_source = BeautifulSoup(t["source"], "lxml").text
    
    VAL_hashtags = [h["text"] for h in t["entities"]["hashtags"]]
    
    VAL_id_user = t["user"]["id_str"]
    VAL_name_user = t["user"]["screen_name"]
    
    VAL_id_recipient = t["in_reply_to_user_id_str"]
    VAL_name_recipient = t["in_reply_to_screen_name"]
    
    VAL_id_mentions = []
    VAL_name_mentions = []
    if t["entities"]["user_mentions"] != []:
        for mention in t["entities"]["user_mentions"]:
            VAL_id_mentions.append(mention["id_str"])
            VAL_name_mentions.append(mention["name"])
        else:
            VAL_id_mentions, VAL_name_mentions = None, None
            
    VAL_geo = None
    if t["geo"] is not None:
        VAL_geo = t["geo"]["coordinates"]
    
    VAL_coords = None
    if t["coordinates"] is not None:
        VAL_coords = t["coordinates"]["coordinates"]
    
    if t["place"] is not None:
        VAL_id_place = t["place"]["id"]
        VAL_name_place = t["place"]["full_name"]
        VAL_country_place = t["place"]["country"]
        #coords_place = t["place"]["bounding_box"]["coordinates"]
    else:
        VAL_id_place, VAL_name_place, VAL_country_place = None, None, None
    
    for e in ["favorite_count", "quote_count", "reply_count", "retweet_count", "retweeted"]:
        if e in t.keys():
            globals()["VAL_"+e] = t[e]
        else:
            globals()["VAL_"+e] = None
            
    tweet_data = [v for v in vars() if v.startswith("VAL_")]
    
    stmt_insert = f"INSERT INTO {schema}.{tbl_name} ({col_names})"
    stmt_values = f"VALUES ({', '.join(['%s'] * len(tweet_data))})"
    
    stmt = " ".join([stmt_insert, stmt_values])
    
    engine.execute(stmt, tuple(globals()[v] for v in tweet_data))

engine.close()